## Vgg16BN Finetuning experienced with the batch_normalized fc layers

### Result: Can't get val_loss to come down below what we had for regular Vgg16
###              Currently unable to rewire the trained dense-layers back with the conv.
###              model to generate outputs.

In [2]:
%load_ext autoreload

In [3]:
%autoreload Executor

In [4]:
%aimport Executor
%aimport Vgg16
%aimport Vgg16BN

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [1]:
from Executor import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
executor = ExecutorBuilder().\
    with_runID("first.bn").\
    and_().\
    with_Vgg16BN().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()


Pretrained Vgg16BN model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [3]:
#done..
executor.precompute_conv_model_outputs()
#executor.load_precomputed_conv_models("")

precomputing conv. model outputs..
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
('models saved to files: ', 'precomputed_trn_features.first.bn.h5', ' and ', 'precomputed_val_features.first.bn.h5')
done.


<Executor.Executor instance at 0x7fcb698e8368>

In [7]:
# executor.tune_softmax_layer_for_epochs(1)
# # 6255/6255 [==============================] - 201s - loss: 2.3536 - acc: 0.4571 - val_loss: 1.1148 - val_acc: 0.5261


In [29]:
# from keras.layers import Dense, BatchNormalization, Dropout, Lambda, Flatten
# from keras.layers.convolutional import MaxPooling2D
# from keras.models import Sequential

# def get_new_fc_model(conv_layer, num_softmax_classes, new_dropout=0.5):
    
#     new_model = Sequential([
#         MaxPooling2D(input_shape=conv_layer.output_shape[1:]),
#         Flatten(),
#         Dense(4096, activation='relu'),
#         BatchNormalization(),
#         Dropout(new_dropout),        
#         Dense(4096, activation='relu'),
#         BatchNormalization(),
#         Dropout(new_dropout),
#         Dense(num_softmax_classes, activation='softmax')
#     ])
        
#     return new_model
# fc_model = get_new_fc_model(executor.conv_layers[-1], 3)

In [31]:
# new_dropout = 0.5
# def proc_wgts(layer, prev_dropout, new_dropout):
#     '''
#     evaluates an array of rescaled weights

#     :param layer: fc layer
#     :param prev_dropout: prev dropout with which the layer was trained
#     :param new_dropout:  new dropout to apply
#     :return: array of rescaled weights
#     '''
#     scale = (1.0 - prev_dropout)/(1.0 - new_dropout)
#     return [o * scale for o in layer.get_weights()]

# for l1, l2 in zip(fc_model.layers, executor.fc_layers):
#     print("found dense layer. Distributing scaled weights..")
#     l1.set_weights(proc_wgts(l2, executor.dropout, new_dropout))


found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..


In [8]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.5)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)


In [9]:
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

for layer in fc_model.layers:
    layer.trainable=False

fc_model.layers[-1].trainable = True

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 6.0104 - acc: 0.4785 - val_loss: 5.8529 - val_acc: 0.5391
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 2.3671 - acc: 0.5965 - val_loss: 2.9775 - val_acc: 0.5842
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 1.2609 - acc: 0.7180 - val_loss: 2.2184 - val_acc: 0.6243
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.8485 - acc: 0.8131 - val_loss: 1.6840 - val_acc: 0.6714
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.5503 - acc: 0.8731 - val_loss: 2.0305 - val_acc: 0.6643


In [11]:
# tune for 5 more times...
fc_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.3444 - acc: 0.9209 - val_loss: 1.6707 - val_acc: 0.6708
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.3201 - acc: 0.9279 - val_loss: 1.5415 - val_acc: 0.6830
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.3524 - acc: 0.9236 - val_loss: 1.5502 - val_acc: 0.6817
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.3085 - acc: 0.9300 - val_loss: 1.5626 - val_acc: 0.6817
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.3336 - acc: 0.9317 - val_loss: 1.4276 - val_acc: 0.6901


In [12]:
# Tune for 5 more times with reduced learn_rate
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.3200 - acc: 0.9285 - val_loss: 1.5457 - val_acc: 0.6875
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.3048 - acc: 0.9330 - val_loss: 1.4755 - val_acc: 0.6875
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.3054 - acc: 0.9330 - val_loss: 1.4563 - val_acc: 0.6882
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.3100 - acc: 0.9354 - val_loss: 1.3830 - val_acc: 0.6914
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.2959 - acc: 0.9364 - val_loss: 1.4089 - val_acc: 0.6882


In [13]:
# Now set the entire dense layers as being trainable, and run the experiments:
for layer in fc_model.layers:
    layer.trainable=True
    
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 0.2994 - acc: 0.9364 - val_loss: 1.3897 - val_acc: 0.6927
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.2710 - acc: 0.9461 - val_loss: 1.3621 - val_acc: 0.6979
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.2344 - acc: 0.9519 - val_loss: 1.3399 - val_acc: 0.7005
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.2348 - acc: 0.9482 - val_loss: 1.3396 - val_acc: 0.6934
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.2241 - acc: 0.9532 - val_loss: 1.3258 - val_acc: 0.6953


In [14]:
# 5 more epochs at same lr
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))


fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 0.2052 - acc: 0.9533 - val_loss: 1.3321 - val_acc: 0.6946
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.2124 - acc: 0.9565 - val_loss: 1.3407 - val_acc: 0.6953
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.1946 - acc: 0.9618 - val_loss: 1.3118 - val_acc: 0.6979
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.1838 - acc: 0.9613 - val_loss: 1.2994 - val_acc: 0.6998
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.1827 - acc: 0.9610 - val_loss: 1.2969 - val_acc: 0.6966


In [15]:
fc_model.save_weights("bn_dense_layers_val_loss=1_29.h5")

In [16]:
# final val_loss increases.. so decrease lr
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 8s - loss: 0.2119 - acc: 0.9538 - val_loss: 1.3929 - val_acc: 0.6992
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.1461 - acc: 0.9680 - val_loss: 1.3732 - val_acc: 0.7108
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.1222 - acc: 0.9747 - val_loss: 1.4829 - val_acc: 0.6953
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.0846 - acc: 0.9835 - val_loss: 1.3959 - val_acc: 0.7088
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.0849 - acc: 0.9816 - val_loss: 1.4390 - val_acc: 0.7063


In [21]:
model =  executor.vgg.con

## Can't seem to get past this !! Will get back to this later!

In [23]:
model.add(fc_model)

IndexError: tuple index out of range